<a href="https://colab.research.google.com/github/mohannishant6/thesis/blob/master/notebooks/traffic_violations_catboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# !pip install catboost
# !pip install category_encoders
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from category_encoders import *
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import accuracy_score, f1_score,jaccard_score,multilabel_confusion_matrix
import lightgbm as lgbm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
import pandas as pd
import numpy as np


In [42]:
df=pd.read_csv('https://data.montgomerycountymd.gov/api/views/4mse-ku6q/rows.csv?accessType=DOWNLOAD')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (19,20,21,22,23,24,25,34) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
df1=df.loc[:,[ 'Description','Belts','Property Damage','Fatal','Commercial License','HAZMAT','Commercial Vehicle','Alcohol','Work Zone','Year','Race','Gender','Arrest Type','Violation Type']].dropna()

X=df1.iloc[:,:-1]
y=df1.iloc[:,-1]

In [49]:
X.shape

(1660444, 13)

In [0]:
#split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.02, random_state=42)

In [53]:
#catboost
# model_cat = CatBoostClassifier(iterations=10000,task_type="GPU", devices='0:1',)
# model_cat.fit(X_train, y_train, verbose=0,cat_features=[1,2,3,4,5,6,7,8,10,11,12],text_features=[0])
# accuracy_score(y_test, model_cat.predict(X_test))

0.8025234123279834

In [56]:
model_cat = CatBoostClassifier(task_type="GPU", devices='0:1')
parameters = {'depth'         : [6,8,10,12,14,16],
              'learning_rate' : [0.01, 0.05, 0.1,.2,.5],
              'iterations'    : [500,1000,2000,5000]
              }
grid = GridSearchCV(estimator=model_cat, param_grid = parameters)
grid.fit(X_train, y_train,verbose=0,cat_features=[1,2,3,4,5,6,7,8,10,11,12],text_features=[0])    

# Results from Grid Search
print("\n========================================================")
print(" Results from Grid Search " )
print("========================================================")    

print("\n The best estimator across ALL searched params:\n",
      grid.best_estimator_)

print("\n The best score across ALL searched params:\n",
      grid.best_score_)


print("\n The best parameters across ALL searched params:\n",
      grid.best_params_)

print("\n ========================================================")

Streaming output truncated to the last 5000 lines.
16:	learn: 0.7078737	total: 2.2s	remaining: 1m 2s
17:	learn: 0.6908858	total: 2.32s	remaining: 1m 2s
18:	learn: 0.6752275	total: 2.45s	remaining: 1m 1s
19:	learn: 0.6604302	total: 2.57s	remaining: 1m 1s
20:	learn: 0.6464902	total: 2.71s	remaining: 1m 1s
21:	learn: 0.6334950	total: 2.81s	remaining: 1m 1s
22:	learn: 0.6213885	total: 2.92s	remaining: 1m
23:	learn: 0.6100247	total: 3.05s	remaining: 1m
24:	learn: 0.5994402	total: 3.17s	remaining: 1m
25:	learn: 0.5895670	total: 3.31s	remaining: 1m
26:	learn: 0.5801496	total: 3.43s	remaining: 1m
27:	learn: 0.5713654	total: 3.58s	remaining: 1m
28:	learn: 0.5631001	total: 3.72s	remaining: 1m
29:	learn: 0.5553110	total: 3.85s	remaining: 1m
30:	learn: 0.5478726	total: 3.99s	remaining: 1m
31:	learn: 0.5408079	total: 4.12s	remaining: 1m
32:	learn: 0.5341683	total: 4.28s	remaining: 1m
33:	learn: 0.5280021	total: 4.41s	remaining: 1m
34:	learn: 0.5219271	total: 4.54s	remaining: 1m
35:	learn: 0.5163448

In [0]:
c=grid.best_estimator_

In [75]:
print("training")
print("macro f1: ",f1_score(y_train,c.predict(X_train), average='macro'))
print("micro f1: ",f1_score(y_train, c.predict(X_train), average='micro'))
print("macro jaccard: ",jaccard_score(y_train,c.predict(X_train), average='macro'))
print("micro jaccard: ",jaccard_score(y_train, c.predict(X_train), average='micro'))

print("test")
print("macro f1: ",f1_score(y_train,c.predict(X_train), average='macro'))
print("micro f1: ",f1_score(y_train, c.predict(X_train), average='micro'))
print("macro jaccard: ",jaccard_score(y_train,c.predict(X_train), average='macro'))
print("micro jaccard: ",jaccard_score(y_train, c.predict(X_train), average='micro'))


macro f1:  0.662885498784336
micro f1:  0.8025767636512243


In [79]:
print(multilabel_confusion_matrix(y_test, c.predict(X_test)))

array([[[ 820843,   78686],
        [ 241663,  486043]],

       [[1548660,     883],
        [     33,   77659]],

       [[1626355,       8],
        [    834,      38]],

       [[ 564593,  241677],
        [  78724,  742241]]])